In [1]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [3]:
from pdf2image import convert_from_path
import pytesseract
pdfs = "./Articles/amz_letters.pdf"
pages = convert_from_path(pdfs, 350)
# print(pages)

i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"  
    page.save(f'AMZ_Letters/{image_name}', "JPEG")
    i = i+1 

In [6]:

# use this command to install open cv2
# pip install opencv-python

# use this command to install PIL
# pip install Pillow

import cv2
from PIL import Image

def mark_region(image_path):
    print(image_path)
    im = cv2.imread(image_path)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_coordinates = []
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)

        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])

        if y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])


    return image, line_items_coordinates

In [9]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
pageNum = 'Page_26'
img = cv2.imread(f'AMZ_Letters/{pageNum}.jpg')

custom_config = r'--psm 3'
# data = pytesseract.image_to_string(img, config=custom_config)
# print(data)
# df = pd.DataFrame(data = data)
# display(df)
# df.head(10)
d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
# print(d.keys())
df = pd.DataFrame(d)
df.to_csv(f'./AMZ{pageNum}trialData.csv')
display(df)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,2975,3850,-1,
1,2,1,1,0,0,0,2399,87,129,38,-1,
2,3,1,1,1,0,0,2399,87,129,38,-1,
3,4,1,1,1,1,0,2399,87,129,38,-1,
4,5,1,1,1,1,1,2399,87,129,38,95,2004
...,...,...,...,...,...,...,...,...,...,...,...,...
446,5,1,10,1,2,2,266,2856,141,34,95,income
447,5,1,10,1,2,3,422,2862,185,28,96,statement
448,5,1,10,1,2,4,621,2856,119,34,96,would
449,5,1,10,1,2,5,753,2856,44,34,96,be


In [10]:
#Grouping the text by their block numbers
processedDf = df.groupby(
   ['block_num']
).agg(
    {
         'left':min,  
         'top': min,  
         'width': max,
         'height': max,
         'text': (lambda x: ' '.join(x))
    }
)

grouped_df = processedDf.reset_index()
grouped_df.to_csv(f'./{pageNum}paraBlocksTrial.csv')

display(grouped_df)
# Creating a visualization that showcases the boxes 
for i in range(len(grouped_df)):
    # print(grouped_df.loc[i,'left'])
#     if int(d['conf'][i]) > 60:
    (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


cv2.imshow('img', img)
cv2.waitKey(5000)

# n_boxes = len(d['text'])
# print(n_boxes)
# for i in range(n_boxes):
#     if int(d['conf'][i]) > 60:
#         (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
#         img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


# cv2.imshow('img', img)
# cv2.waitKey(5000)


,block_num,left,top,width,height,text
0,0,0,0,2975,3850,
1,1,2399,87,129,38,2004
2,2,929,365,1033,198,amazon.com A
3,3,139,626,2316,138,To our shareholders: Our ultimate financia...
4,4,139,815,2598,205,"Why not focus first and foremost, as many d..."
5,5,139,1070,2607,141,"Though some may find it counterintuitive, a..."
6,6,139,1272,2609,196,To illustrate with a hypothetical and very ...
7,7,139,1529,2547,204,Continue to imagine that business is boomin...
8,8,244,1784,1285,44,Here are the income statements for the firs...
9,9,400,1881,2088,861,Earnings Year 1 Year 2 Year 3 Year 4 (in ...


-1

In [4]:
import cv2
import pytesseract

filename = 'Page_1.jpg'

# read the image and get the dimensions
img = cv2.imread(filename)
h, w, _ = img.shape # assumes color image
# run tesseract, returning the bounding boxes
boxes = pytesseract.image_to_boxes(img) # also include any config options you use

# draw the bounding boxes on the image
for b in boxes.splitlines():
    b = b.split(' ')
    img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)

# show annotated image and wait for keypress
cv2.imshow(filename, img)
cv2.waitKey(0)

111

In [2]:
import cv2
import numpy as np

# Load image, grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('Page_1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Create rectangular structuring element and dilate
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.dilate(thresh, kernel, iterations=4)

# Find contours and draw rectangle
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)

cv2.imshow('thresh', thresh)
cv2.imshow('dilate', dilate)
# cv2.imshow('image', image)
cv2.waitKey(5000)

-1